In [1]:
import sqlite3
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import json

In [2]:
SQLITE_DB_PATH = "HistoricalQA_DB.sqlite3"

In [5]:
# Connect to SQLite database
conn = sqlite3.connect(SQLITE_DB_PATH)
cursor = conn.cursor()

# Read answers from json file
answers = open("all_answers.json", "r")
answers = json.load(answers)

# load model for calculating sentence embeddings
tokenizer = AutoTokenizer.from_pretrained('/Users/lipeihong/Desktop/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('/Users/lipeihong/Desktop/all-MiniLM-L6-v2')


In [34]:
# Function to calculate sentence embeddings in bytes
def get_wordvector(text):
    inputs = tokenizer(text, return_tensors="pt")
    model_out = model(**inputs)
    embeddings = model_out.last_hidden_state[:, 0, :]
    embeddings_np = embeddings.detach().numpy()
    return embeddings_np.tobytes()

In [38]:
# append the answers as well as it's characteristic(embedding) to the database
for answer in answers:
    vector = get_wordvector(answer)
    # add the answer to the database
    cursor.execute("INSERT INTO HistoricalQA (answer_text) VALUES (?)", (answer,))
    answer_id = cursor.lastrowid
    # add the embedding to the database
    cursor.execute("INSERT INTO answer_embeddings (answer_id, embedding) VALUES (?, ?)",
                   (answer_id, vector))

In [ ]:
# Save (commit) the changes and close the connection
conn.commit()
conn.close()